In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pymssql

In [41]:
positions = pd.read_csv("files/OpenPosition_3_19_2022.csv")
positions = positions[["Symbol", "LastPrice"]]
positions.columns = ["ticker", "price"]
positions = positions.set_index("ticker")
positions = positions.squeeze()

In [42]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\pool\base.py", line 753, in _finalize_fairy
    fairy._reset(pool)
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\pool\base.py", line 1004, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\dialects\mssql\base.py", line 2792, in do_rollback
    super(MSDialect, self).do_rollback(dbapi_connection)
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\sqlalchemy\engine\default.py", line 683, in do_rollback
    dbapi_connection.rollback()
  File "src\pymssql\_pymssql.pyx", line 316, in pymssql._pymssql.Connection.rollback
  File "src\pymssql\_pymssql.pyx", line 300, in pymssql._pymssql.Connection.rollback
  File "src\pymssql\_mssql.pyx", line 1055, in pymssql._mssql.MSSQLCo

In [25]:
x = pd.read_sql("select max(date) from sep", conn)
maxdate= x.astype(str).iloc[0, 0]
string = "select ticker, close_ from sep where date=" + "'" + maxdate + "'"
prices = pd.read_sql(string, conn)
prices.columns = ["ticker", "price"]
prices = prices.set_index("ticker")
prices = prices.squeeze()

In [28]:
prices = pd.concat((positions, prices))
duplicates = prices.index.duplicated()

In [35]:
dups = prices[duplicates].index.to_list()
lst = [x for x in positions.index if x not in dups]

In [40]:
positions = pd.read_csv("files/OpenPosition_3_17_2022.csv", index_col="Symbol")
positions = positions.drop(index=lst)
positions.to_csv("files/OpenPosition_3_19_2022.csv")

In [31]:
prices[~duplicates]

ticker
AEYE         6.7700
AIT        102.0800
ALLK         5.9800
ALSN        39.5250
ALX        249.1501
             ...   
SES.WS       1.0500
SFE          3.7700
SLNAW        0.2400
SPRU         1.0900
STRY.WS      0.0190
Name: price, Length: 7671, dtype: float64

In [22]:
len([x for x in prices.index if x in positions.index])

197

In [23]:
len(prices[~prices.index.duplicated()])

7671

In [26]:
len(prices)

7664

In [27]:
len(positions)

102

In [ ]:
len()

### Update portfolio

In [23]:
ranks = pd.read_excel("files/ranks1.xlsx", index_col=0)
ranks = ranks.squeeze()

prices = pd.read_excel("files/prices1.xlsx", index_col=0)
prices = prices.squeeze()

equity = cash + (positions*prices).sum()
weights = (positions*prices) / equity

sellall = (ranks>10) & (positions !=0)
sellallamounts = positions
sellsome = (ranks<=10) & (weights>0.2)
sellsomeamounts = (equity*(weights-0.2)/prices).astype(int)

trades = - np.where(sellall, sellallamounts, 0)
trades -= np.where(sellsome, sellsomeamounts, 0)
trades = pd.Series(trades, index=ranks.index)

cash -= (trades*prices).sum()

tickers = ranks[positions==0].sort_values().index.to_list()

while cash >= 500:
    tick = tickers.pop(0)
    dollars = min(cash, weight*equity)
    trades[tick] = int(dollars/prices[tick])
    cash -= trades[tick]*prices[tick]


positions += trades